In [2]:
# Create CNN

In [14]:
# Importing the libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import (
    Conv2D, MaxPooling2D, Flatten,
    Dense, Dropout, Input
)
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
from skimage.io import imshow
from os.path import join
import glob
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

In [18]:
# Connect to google
from google.colab import drive
drive.mount('/content/gdrive')

# Get the preprocessed train and test data
training_data = pd.read_csv('/content/gdrive/MyDrive/training.csv')
test_data = pd.read_csv('/content/gdrive/MyDrive/test.csv')

Mounted at /content/gdrive


In [19]:
# Define the architecture
def create_model():
    '''
    Define a CNN where input to the model must be a 96 by 96 pixel, greyscale, image
    -------
    Returns:
    -------
    model: A fully-connected output layer with 30 facial keypoints
    '''

    model = Sequential()
    model.add(layers.Conv2D(32, (5,5), activation='relu', input_shape=(96, 96, 1)))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.1))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())

    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(30))
    return model

In [20]:
# Compile the model
def compile_model(model, optimizer, loss, metrics):
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [21]:
# Fit the model
def train_model(model, X_train, y_train):
    return model.fit(X_train, y_train, epochs=100, batch_size=200, verbose=1, validation_split=0.2)

In [22]:
# Load weights for a previously trained model
def load_trained_model(model):
    model.load_weights('weights/checkpoint-300.hdf5')

In [25]:
# Testing the model
def test_model(model):    
    data_path = join('','*g')
    files = glob.glob(data_path)
    # Test model performance on a screenshot for the webcam
    for i,f1 in enumerate(files):    
        if f1 == 'Capture.PNG':
            img = imread(f1)
            # Convert RGB image to grayscale
            img = rgb2gray(img)
            # Resize to an array of size 96x96
            test_img = resize(img, (96,96))
      test_img = np.array(test_img)
      # Model takes input of shape = [batch_size, height, width, no. of channels]
      test_img_input = np.reshape(test_img, (1,96,96,1))
      # shape = [batch_size, values]
      prediction = model.predict(test_img_input)
      visualize_points(test_img, prediction[0])


      # Test on first 10 samples of the test set
      for i in range(len(imgs_test)):
            # Model takes input of shape = [batch_size, height, width, no. of channels]
            test_img_input = np.reshape(imgs_test[i], (1,96,96,1))
            # shape = [batch_size, values]
            visualize_points(imgs_test[i], prediction[0])
            if i == 10:
                break      



                      

IndentationError: ignored

In [ ]:
# Train the model
model = get_model()
compile_model(model)
#train_model(model)
load_trained_model(model)
test_model(model)

In [1]:
# Read data

In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from skimage.io import imshow
import math

In [ ]:
# Check if row has any NaN values 
def has_nan(keypoints):
      for i in range(len(keypoints)):
          if math.isnan(keypoints[i]):
              return True
      return False

In [ ]:
# Function which plots an image with it's corresponding keypoints
def visualize_points(img, points):
      fig,ax = plt.subplots(1)
      ax.set_aspect('equal')
      imshow(img)
      for i in range(0,len(points),2):
          # Denormalize x-coordinate
          x_renorm = (points[i]+0.5)*96
          # Denormalize y-coordinate
          y_renorm = (points[i+1]+0.5)*96
          # Plot the keypoints at the x and y coordinates
          circ = Circle((x_renorm, y_renorm),1, color='r')
          ax.add_patch(circ)
      plt.show()

In [ ]:
# Read the data as Dataframes
training = pd.read_csv('data/training.csv')
test = pd.read_csv('data/test.csv')

In [ ]:
# Get training data
imgs_train = []
points_train = []
for i in range(len(training)):
    points = training.iloc[i,:-1]
    if has_nan(points) is False:
        # Get the image data
        test_image = training.iloc[i,-1] 
        test_image = np.array(test_image.split(' ')).astype(int)
        # Reshape into an array of size 96x96
        test_image = np.reshape(test_image, (96,96))
        # Normalize image
        test_image = test_image/255
        imgs_train.append(test_image)

        keypoints = training.iloc[i,:-1].astype(int).values
        # Normalize keypoint coordinates
        keypoints = keypoints/96 - 0.5  
        points_train.append(keypoints)

imgs_train = np.array(imgs_train)    
points_train = np.array(points_train)  

In [ ]:
# Get test data
imgs_test = []
for i in range(len(test)):
    # Get the image data
    test_image = test.iloc[i,-1]
    test_image = np.array(test_image.split(' ')).astype(int)
    # Reshape into an array of size 96x96
    test_image = np.reshape(test_image, (96,96))        
    # Normalize image
    test_image = test_image/255     
    imgs_test.append(test_image)
    
imgs_test = np.array(imgs_test)

In [ ]:
# Data Augmentation by mirroring the images
def augment(img, points):
      # Mirror the image
      f_img = img[:, ::-1]
      # Mirror the key point coordinates
      for i in range(0,len(points),2):
          # Denormalize x-coordinate
          x_renorm = (points[i]+0.5)*96
          # Get distance to midpoint
          x_renorm_flipped = x_renorm - 2*dx
          # Normalize x-coordinate
          points[i] = x_renorm_flipped/96 - 0.5
      return f_img, points

aug_imgs_train = []
aug_points_train = []
for i, img in enumerate(imgs_train):
      f_img, f_points = augment(img, points_train[i])
      aug_imgs_train.append(f_img)
      aug_points_train.append(f_points)

aug_imgs_train = np.array(aug_imgs_train)
aug_points_train = np.array(aug_points_train)       

In [ ]:
# Combine the original data and augmented data
imgs_total = np.concatenate((imgs_train, aug_imgs_train), axis=0)       
points_total = np.concatenate((points_train, aug_points_train), axis=0)

In [ ]:
def get_train_data():
    imgs_total_reshaped = np.reshape(imgs_total, (imgs_total.shape[0],imgs_total.shape[1],imgs_total.shape[2], 1))
    return imgs_total_reshaped,points_total

In [ ]:
def get_test_data():
    return imgs_test

In [ ]:
## Implement the model in real-time

In [ ]:
# Importing the libraries
import numpy as np
from training import get_model, load_trained_model, compile_model
import cv2

In [ ]:
# Load the trained model
model = get_model()
compile_model(model)
load_trained_model(model)

In [ ]:
# Get frontal face haar cascade
face_cascade = cv2.CascadeClassifier('cascades/haarcascade_frontalface_default.xml')

In [ ]:
# Get webcam
camera = cv2.VideoCapture(0)

In [ ]:
# Run the program infinitely
while True:
    # Read data from the webcam
    grab_trueorfalse, img = camera.read()

    # Preprocess input fram webcam & Convert RGB data to Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Identify faces in the webcam        
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # For each detected face using tha Haar cascade
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        img_copy = np.copy(img)
        img_copy_1 = np.copy(img)
        roi_color = img_copy_1[y:y+h, x:x+w]

        # Width of region where face is detected
        width_original = roi_gray.shape[1]
        # Height of region where face is detected
        height_original = roi_gray.shape[0]
        # Resize image to size 96x96
        img_gray = cv2.resize(roi_gray, (96, 96))
        # Normalize the image data 
        img_gray = img_gray/255

        # Model takes input of shape = [batch_size, height, width, no. of channels]
        img_model = np.reshape(img_gray, (1,96,96,1))
        # Predict keypoints for the current input
        keypoints = model.predict(img_model)[0]

        # Keypoints are saved as (x1, y1, x2, y2, ......)
        # Read alternate elements starting from index 0
        x_coords = keypoints[0::2]
        # Read alternate elements starting from index 1
        y_coords = keypoints[1::2]

        # Denormalize x-coordinate
        x_coords_denormalized = (x_coords+0.5)*width_original
        # Denormalize y-coordinate
        y_coords_denormalized = (y_coords+0.5)*height_original

        # Plot the keypoints at the x and y coordinates
        for i in range(len(x_coords)):
             cv2.circle(roi_color, (x_coords_denormalized[i], y_coords_denormalized[i]), 2, (255,255,0), -1)

        # Particular keypoints for scaling and positioning of the filter
        left_lip_coords = (int(x_coords_denormalized[11]), int(y_coords_denormalized[11]))
        right_lip_coords = (int(x_coords_denormalized[12]), int(y_coords_denormalized[12]))
        top_lip_coords = (int(x_coords_denormalized[13]), int(y_coords_denormalized[13]))
        bottom_lip_coords = (int(x_coords_denormalized[14]), int(y_coords_denormalized[14]))
        left_eye_coords = (int(x_coords_denormalized[3]), int(y_coords_denormalized[3]))
        right_eye_coords = (int(x_coords_denormalized[5]), int(y_coords_denormalized[5]))
        brow_coords = (int(x_coords_denormalized[6]), int(y_coords_denormalized[6]))

        # Scale filter according to keypoint coordinates
        beard_width = right_lip_coords[0] - left_lip_coords[0]
        glasses_width = right_eye_coords[0] - left_eye_coords[0]

        # Used for transparency overlay of filter using the alpha channel
        img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2BGRA)

        # Sunlasses filter
        sunglasses = cv2.imread('filters/sunglasses.png', -1)
        sunglasses = cv2.resize(sunglasses, (sunglasses_width*2,150))
        sgw,sgh,sgc = sunglasses.shape
        
        # Overlay the filter based on the alpha channel
        for i in range(0,sgw):       
            for j in range(0,sgh):
                if sunglasses[i,j][3] != 0:
                    img_copy[brow_coords[1]+i+y-50, left_eye_coords[0]+j+x-60] = sunglasses[i,j]

        # Rainbow filter
        rainbow = cv2.imread('filters/rainbow.png', -1)
        rainbow = cv2.resize(rainbow, (rainbow_width* _ ))
        rbw,rbh, rbc = rainbow.shape

        # Overlay the filter based on the alpha channel
        for i in range(0,rbw):
            for j in range(0,rbh):
                 if rainbow[i,j][3] != 0:
                       img_copy[brow_coords[1]+i+y-50, left_eye_coords[0]+j+x-60] = rainbow[i,j]

        # Glasses filter
        glasses = cv2.imread('filters/glasses.png', -1)
        glasses = cv2.resize(sunglasses, (glasses_width*2,150))
        gw,gh,gc = glasses.shape
        
        # Overlay the filter based on the alpha channel
        for i in range(0,gw):       
            for j in range(0,gh):
                if glasses[i,j][3] != 0:
                    img_copy[brow_coords[1]+i+y-50, left_eye_coords[0]+j+x-60] = glasses[i,j]

        
        # Demon mask filter
        demon =  cv2.imread('filters/demon.png', -1)
        demon = cv2.resize(demon, (demon_width*2,150))
        dw,dh,dc = demon.shape
        
        # Overlay the filter based on the alpha channel
        for i in range(0,dw):       
            for j in range(0,dh):
                if demon[i,j][3] != 0:
                    img_copy[brow_coords[1]+i+y-50, left_eye_coords[0]+j+x-60] = demon[i,j]

        
        # Doc Dog filter
        doc_dog =  cv2.imread('filters/doc_dog.png', -1)
        doc_dog = cv2.resize(doc_dog, (doc_dog_width*2,150))
        ddw,ddh,ddc = doc_dog.shape
        
        # Overlay the filter based on the alpha channel
        for i in range(0,ddw):       
            for j in range(0,ddh):
                if doc_dog[i,j][3] != 0:
                    img_copy[brow_coords[1]+i+y-50, left_eye_coords[0]+j+x-60] = doc_dog[i,j]


        # King Wig filter
        king_wig =  cv2.imread('filters/king_wig.png', -1)
        king_wig = cv2.resize(king_wig, (king_wig_width*2,150))
        kww,kwh,kwc = king_wig.shape
        
        # Overlay the filter based on the alpha channel
        for i in range(0,kww):       
            for j in range(0,kwh):
                if king_wig[i,j][3] != 0:
                    img_copy[brow_coords[1]+i+y-50, left_eye_coords[0]+j+x-60] = king_wig[i,j]


        # Top Hat filter
        top_hat =  cv2.imread('filters/top_hat.png', -1)
        top_hat = cv2.resize(top_hat, (top_hat_width*2,150))
        thw,thh,thc = top_hat.shape
        
        # Overlay the filter based on the alpha channel
        for i in range(0,thw):       
            for j in range(0,thh):
                if top_hat[i,j][3] != 0:
                    img_copy[brow_coords[1]+i+y-50, left_eye_coords[0]+j+x-60] = top_hat[i,j]

        # Dalmatian filter
        dalmatian =  cv2.imread('filters/dalmatian.png', -1)
        dalmatian = cv2.resize(dalmatian, (dalmatian_width*2,150))
        daw,dah,dac = dalmatian.shape
        
        # Overlay the filter based on the alpha channel
        for i in range(0,daw):       
            for j in range(0,dah):
                if dalmatian[i,j][3] != 0:
                    img_copy[brow_coords[1]+i+y-50, left_eye_coords[0]+j+x-60] = dalmatian[i,j]


         # Revert back to background
         img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGRA2BGR)

         # Output with the filter placed on the face
         cv2.imshow('Output',img_copy)

         # Place keypoints on the webcam input
         cv2.imshow('Keypoints predicted',img_copy_1)

      # Original webcame Input
      cv2.imshow('Webcam',img)

      # If 'f' is pressed, stop reading and break the loop
      if cv2.waitKey(1) & 0xFF == ord("f"):
          break
